In [339]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--enable-javascript')
chrome_options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36')

from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from lxml import etree, html
import json


class fotocasa_scraping:
  '''
  Clase para inicializar el scraping de Fotocasa.com
  '''

  def parse_properties(self, driver, url_list):
    # Recibo una lista de urls de la propiedad y lo separo en un diccionario. Devuelvo el diccionario.
    
    df_page = pd.DataFrame()
    
    for url in url_list:
        driver.get(url)
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt,'html.parser')
        prop_scripts = soup.findAll('script')
        prop_features = ''.join([re.search('window.__INITIAL_PROPS__ = JSON.parse(.*)\n',str(x)).group(1) for x in prop_scripts if re.search('window.__INITIAL_PROPS__',str(x))])
        prop_features_clean = re.sub(r'\\"','"',prop_features)
        prop_features_clean = re.sub(r'\\\\"','',prop_features_clean)
        prop_features_clean = re.sub(r'\("|"\);','',prop_features_clean)
        prop_data = json.loads(prop_features_clean)
        
        #posibilidad de crear una función de comprobación de los campos para asignar valores default en caso de que no existan

        realestate = {
            'title': prop_data['propertyTitle'],
            'country': prop_data['realEstate']['address']['country'],
            'district': prop_data['realEstate']['address']['district'],
            'neighborhood': prop_data['realEstate']['address']['neighborhood'],
            'street': prop_data['realEstate']['location'],
            'zipCode': prop_data['realEstate']['address']['zipCode'],
            'province': prop_data['realEstate']['address']['province'],
            'buldingType': prop_data['realEstate']['buildingType'],
            'client': prop_data['realEstate']['clientAlias'],
            'latitude': prop_data['realEstate']['coordinates']['latitude'],
            'longitude': prop_data['realEstate']['coordinates']['longitude'],
            'isNewConstruction': prop_data['realEstate']['isNewConstruction'],
            'rooms': prop_data['realEstate']['features']['rooms'],
            'bathrooms': prop_data['realEstate']['features']['bathrooms'],
            'surface': prop_data['realEstate']['features']['surface'],
            #'energyCertificate': prop_data['realEstate']['propertyEnergyCertificate']['energyEfficiencyRatingType'],
            'hotWater': prop_data['realEstate']['features']['hotWater'],
            'heating': prop_data['realEstate']['features']['heating'],
            'conservationState': prop_data['realEstate']['features']['conservationState'],
            'antiquity': prop_data['realEstate']['features']['antiquity'],
            'floor': prop_data['realEstate']['features']['floor'],
            'surfaceLand': prop_data['realEstate']['features']['surfaceLand'],
            #'otherFeatures': prop_data['realEstate']['otherFeatures'],
            'price': prop_data['realEstate']['price']
        }

        df = pd.DataFrame([realestate])
        df_page = pd.concat([df_page,df],ignore_index=True)
        
    return df_page

  def property_list(self, driver, city, page):
    # Recibo un número de página. Almaceno las urls de todas las propiedades de cada página de parrilla. Devuelvo una lista de urls.
    driver.get('https://www.fotocasa.es/es/comprar/viviendas/' + city.lower() + '-provincia/todas-las-zonas/l' + '/' + str(page))

    # Creación de lista de article (propiedades) del grid
    properties_grid = []
    while len(properties_grid) < 30: # número de tarjetas de propiedad por página
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt,'html.parser')
        properties_grid = soup.findAll('article')
        ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
        time.sleep(0.2)
    
    # Creación de lista con urls de todas las propiedades del grid
    property_url_list = []
    domain = 'https://www.fotocasa.es'
    for article in properties_grid:
        prop = etree.HTML(str(article))
        try:
            url = prop.xpath('//a[contains(@class, "info-container") or contains(@class, "carousel")]//@href')[0]
            property_url_list.append(f'{domain}{url}')
        except:
            pass


    return property_url_list

  def pages_to_scrape(self, driver, city):
    # Obtengo el número de páginas totales que debo recorrer. Devuelvo un entero.

    driver.get('https://www.fotocasa.es/es/comprar/viviendas/' + city.lower() + '-provincia/todas-las-zonas/l')
    
    page_selector = []
    while len(page_selector) < 1:
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt,'html.parser')
        page_selector = soup.findAll('li',attrs={'class':'sui-MoleculePagination-item'})
        ActionChains(driver).key_down(Keys.PAGE_DOWN).perform()
        time.sleep(0.2)
       
    n_pages = re.search('<span class="sui-AtomButton-inner">(.*)</span>',str(page_selector[-2])).group(1)
    
    return int(n_pages)

  def __init__(self, city='Madrid'):
    '''
    Inicio de la clase con el scraping de Fotocasa para la ciudad indicada como parámetro.
    Por defecto: Madrid
    '''
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
    #n_pages = self.pages_to_scrape(driver, city)
    n_pages = 3
    data = pd.DataFrame()
    
    for page in range(1,n_pages+1):
        print(page)
        properties_per_page = self.property_list(driver, city, page)
        property_data = self.parse_properties(driver, properties_per_page)
        data = pd.concat([data,property_data], ignore_index=True)
        
    print(data.shape)
    print(data)

    # Leer página inicial de la parrilla de la ciudad enviada por parámetro. OK
    # Obtener el número de páginas totales a recorrer. OK
    # Almacenar todas las urls de propiedades que debo recorrer OK
    # Meterme en cada url almacenada y scrapear window.__INITIAL_PROPS__
    # Limpiar el JSON de cada propiedad
    # Cambiar de página y comenzar el proceso iterativo recorriendo todas las propiedades de cada página.

    return None

In [340]:
ft_scraping = fotocasa_scraping()


1
2
(59, 22)
                                                title country  \
0                     Piso en venta en Monte Esquinza  España   
1                                       Piso en venta  España   
2                                       Piso en venta  España   
3                                       Piso en venta  España   
4                                       Piso en venta  España   
5                                       Piso en venta  España   
6                     Piso en venta en Claudio Coello  España   
7                                       Piso en venta  España   
8                                      Ático en venta  España   
9                                       Piso en venta  España   
10                                      Piso en venta  España   
11                                      Piso en venta  España   
12                                      Piso en venta  España   
13                                      Piso en venta  España   
14         